In [6]:
import pandas as pd
from datetime import datetime, date

In [32]:
df = pd.read_csv('../data/transactions.csv', parse_dates=['Date'])

In [33]:
df = df.loc[df['Date'] >= datetime(2021, 1, 1)]

In [34]:
df = df.set_index('Date')

In [35]:
monthly = round(df.resample('M')[['Amount']].sum())
monthly.to_csv('static_data/monthly_spending.csv')
monthly['quartile'] = 'bottom'
monthly

,Amount,quartile
Date,,
2021-01-31,19639.0,bottom
2021-02-28,21191.0,bottom
2021-03-31,20187.0,bottom
2021-04-30,25724.0,bottom
2021-05-31,19646.0,bottom
2021-06-30,23407.0,bottom
2021-07-31,17332.0,bottom
2021-08-31,28058.0,bottom
2021-09-30,29742.0,bottom


In [31]:
df

,Description,Original Description,Amount,Transaction Type,Category,Account Name,Labels,Notes
Date,,,,,,,,
